![pageindex_banner](https://pageindex.ai/static/images/pageindex_banner.jpg)

<p align="center">Reasoning-based RAG&nbsp; ◦ &nbsp;No Vector DB&nbsp; ◦ &nbsp;No Chunking&nbsp; ◦ &nbsp;Human-like Retrieval</p>

<p align="center">
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://chat.pageindex.ai">🖥️ Platform</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>&nbsp;
</p>

<div align="center">

[![Star us on GitHub](https://img.shields.io/github/stars/VectifyAI/PageIndex?style=for-the-badge&logo=github&label=⭐️%20Star%20Us)](https://github.com/VectifyAI/PageIndex) &nbsp;&nbsp; [![Follow us on X](https://img.shields.io/badge/Follow%20Us-000000?style=for-the-badge&logo=x&logoColor=white)](https://twitter.com/VectifyAI)

</div>

---


# Agentic Retrieval with PageIndex Chat API

Similarity-based RAG based on Vector-DB has shown big limitations in recent AI applications, reasoning-based or agentic retrieval has become important in current developments. However, unlike classic RAG pipeine with embedding input, top-K chunks returns, re-rank, what should a agentic-native retreival API looks like?

For an agentic-native retrieval system, we need the ability to prompt for retrieval just as naturally as you interact with ChatGPT. Below, we provide an example of how the PageIndex Chat API enables this style of prompt-driven retrieval.


## PageIndex Chat API
[PageIndex Chat](https://chat.pageindex.ai/) is a AI assistant that allow you chat with multiple super-long documents without worrying about limited context or context rot problem. It is based on [PageIndex](https://pageindex.ai/blog/pageindex-intro), a vectorless reasoning-based RAG framework which gives more transparent and reliable results like a human expert.
<div align="center">
  <img src="https://docs.pageindex.ai/images/cookbook/vectorless-rag.png" width="70%">
</div>

You can now access PageIndex Chat with API or SDK.

## 📝 Notebook Overview

This notebook demonstrates a simple, minimal example of agentic retrieval with PageIndex. You will learn:
- [x] How to use PageIndex Chat API.
- [x] How to prompt the PageIndex Chat to make it a retrieval system

### Install PageIndex SDK

In [36]:
%pip install -q --upgrade pageindex

### Setup PageIndex

In [60]:
from pageindex import PageIndexClient

# Get your PageIndex API key from https://dash.pageindex.ai/api-keys
PAGEINDEX_API_KEY = "YOUR_PAGEINDEX_API_KEY"
pi_client = PageIndexClient(api_key=PAGEINDEX_API_KEY)

### Upload a document

In [39]:
import os, requests

pdf_url = "https://arxiv.org/pdf/2507.13334.pdf"
pdf_path = os.path.join("../data", pdf_url.split('/')[-1])
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

response = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(response.content)
print(f"Downloaded {pdf_url}")

doc_id = pi_client.submit_document(pdf_path)["doc_id"]
print('Document Submitted:', doc_id)

Downloaded https://arxiv.org/pdf/2507.13334.pdf
Document Submitted: pi-cmi34m6jy01sg0bqzofch62n8


### Check the processing status

In [61]:
from pprint import pprint

doc_info = pi_client.get_document(doc_id)
pprint(doc_info)

if doc_info['status'] == 'completed':
  print(f"\n Document ready! ({doc_info['pageNum']} pages)")
elif doc_info['status'] == 'processing':
  print("\n Document is still processing. Please wait and check again.")

{'createdAt': '2025-11-16T08:36:41.177000',
 'description': 'This survey provides a comprehensive overview and taxonomy of '
                'Context Engineering for Large Language Models, covering '
                'foundational components, system implementations, evaluation '
                'methods, and future research directions.',
 'id': 'pi-cmi1gp1hg01t20do2l3bgzwz1',
 'name': '2507.13334_19.pdf',
 'pageNum': 166,
 'status': 'completed'}

 Document ready! (166 pages)


### Ask a question about this document

In [55]:
query = "What are the evaluation methods used in this paper?"

for chunk in pi_client.chat_completions(
    messages=[{"role": "user", "content": query}],
    doc_id=doc_id,
    stream=True
):
    print(chunk, end='', flush=True)

I'll help you find the evaluation methods used in this paper. Let me start by examining the document structure to locate the relevant sections.{"doc_name": "2507.13334_19.pdf"}Perfect! I can see there's a dedicated section on "Evaluation" (node_id: 0015) that covers pages 45-50. Let me extract the content from those pages to get detailed information about the evaluation methods.{"doc_name": "2507.13334_19.pdf", "pages": "45-50"}Based on the comprehensive evaluation section of the paper, here are the **evaluation methods** used:

## Evaluation Framework Overview

The paper presents a comprehensive evaluation framework organized into **Component-Level Assessment** and **System-Level Integration Assessment**.

### 1. **Component-Level Assessment (Intrinsic Evaluation)**

#### Prompt Engineering Evaluation:
- **Semantic similarity metrics**
- **Response quality assessment**
- **Robustness testing** across diverse input variations
- **Contextual calibration** assessment

#### Long Context P

## Agentic Retrieval with PageIndex Chat API

You can now easily prompt the PageIndex Chat API to be a retrieval assistant.

In [62]:
retrieval_prompt = f"""
Your job is to retrieve the raw relevant content from the document based on the user's query.

Query: {query}

Return in JSON format:
```json
[
  {{
    "page": <number>,
    "content": "<raw text>"
  }},
  ...
]
```
"""

full_response = ""

for chunk in pi_client.chat_completions(
    messages=[{"role": "user", "content": retrieval_prompt}],
    doc_id=doc_id,
    stream=True
):
    print(chunk, end='', flush=True)
    full_response += chunk

I'll help you find the evaluation methods discussed in this paper. Let me start by examining the document structure to locate relevant sections.{"doc_name": "2507.13334_22.pdf"}Now I can see the document structure. Section 6 is dedicated to "Evaluation" and spans pages 45-50. Let me retrieve the content from those pages to get the detailed evaluation methods.{"doc_name": "2507.13334_22.pdf", "pages": "45-50"}Perfect! I have retrieved the comprehensive evaluation section. Now let me compile the evaluation methods in the requested JSON format.

```json
[
  {
    "page": 45,
    "content": "## 6. Evaluation\n\nThe evaluation of context-engineered systems presents unprecedented challenges that transcend traditional language model assessment paradigms. These systems exhibit complex, multi-component architectures with dynamic, context-dependent behaviors requiring comprehensive evaluation frameworks that assess component-level diagnostics, task-based performance, and overall system robustnes

### Exctarct the JSON retreived results

In [59]:
%pip install -q jsonextractor

def extract_json(content):
    from json_extractor import JsonExtractor
    start_idx = content.find("```json")
    if start_idx != -1:
        start_idx += 7  # Adjust index to start after the delimiter
        end_idx = content.rfind("```")
        json_content = content[start_idx:end_idx].strip()
    return JsonExtractor.extract_valid_json(json_content)

from pprint import pprint
pprint(extract_json(full_response))

Note: you may need to restart the kernel to use updated packages.
[{'content': '## 6. Evaluation\n'
             '\n'
             'The evaluation of context-engineered systems presents '
             'unprecedented challenges that transcend traditional language '
             'model assessment paradigms. These systems exhibit complex, '
             'multi-component architectures with dynamic, context-dependent '
             'behaviors requiring comprehensive evaluation frameworks that '
             'assess component-level diagnostics, task-based performance, and '
             'overall system robustness [841, 1141].\n'
             '\n'
             'The heterogeneous nature of context engineering '
             'components-spanning retrieval mechanisms, memory systems, '
             'reasoning chains, and multi-agent coordination-demands '
             'evaluation methodologies that can capture both individual '
             'component effectiveness and emergent system-level beha